# Graphene size averaging

In [ ]:
import numpy as np
from dplus.CalculationInput import CalculationInput
from dplus.CalculationRunner import EmbeddedLocalRunner
from dplus.DataModels import ManualSymmetry
from dplus.DataModels.models import EPDB
from dplus.CalculationResult import CalculationResult as CR
import dplus.g_r as g
import matplotlib
import matplotlib.colors as colors

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

Defining the parameters

In [ ]:
a = 0.246 ## nm
d_spacing = 0.348  ## nm - according to DOI:10.1038/s41598-018-28402-0
transpose_val = 0.142028166220648
xz = a/2 * np.array([[1, 0, np.sqrt(3)], [0, 1, 0], [1, 0, -np.sqrt(3)]])
sizes = np.linspace(15, 26, 12, dtype=int)

q_min = 0
q_max = 70
grid_size = 150
orientation_method = 'Monte Carlo (Mersenne Twister)'  # "Adaptive (VEGAS) Monte Carlo"  #
use_grid = True
generated_points = q_max * 100
conv = 1e-3
orientation_iterations = 1e6
apply_resolution = False

generated_points_2D = 256
qp = np.linspace(-q_max, q_max, generated_points_2D)

I_2D_mono = np.zeros((len(qp), len(qp)))
I_2D_bi = np.zeros((len(qp), len(qp)))
I_2D_quad = np.zeros((len(qp), len(qp)))


Preparing the graphene lattice

In [ ]:
C_file = r'.\Graphene\C.pdb'
Carbon = EPDB(C_file)
Carbon.centered = True
Carbon.extra_params.solvent_method.value = 0

graphene_unit = ManualSymmetry()
graphene_unit.children.append(Carbon)
graphene_unit.use_grid = True #False

graphene_unit_2 = ManualSymmetry()
graphene_unit_2.add_layer()
graphene_unit_2.add_layer()
graphene_unit_2.layer_params[1].z.value = transpose_val
graphene_unit_2.children.append(graphene_unit)
graphene_unit_2.use_grid = True

graphene_2_layer = ManualSymmetry()
graphene_2_layer.add_layer()
graphene_2_layer.add_layer()
graphene_2_layer.layer_params[1].y.value = d_spacing
graphene_2_layer.layer_params[1].beta.value = 5
graphene_2_layer.children.append(graphene_unit_2)
graphene_2_layer.use_grid = True

graphene_4_layer = ManualSymmetry()
graphene_4_layer.add_layer()
graphene_4_layer.add_layer()
graphene_4_layer.layer_params[1].y.value = 2*d_spacing
graphene_4_layer.layer_params[1].beta.value = 30
graphene_4_layer.children.append(graphene_2_layer)
graphene_4_layer.use_grid = True

Preparing the calculation input

In [ ]:
Graphene = CalculationInput()
Graphene.Domain.populations[0].add_model(graphene_4_layer)
Graphene.DomainPreferences.q_max = q_max
Graphene.DomainPreferences.grid_size = grid_size
Graphene.DomainPreferences.orientation_method = orientation_method
Graphene.DomainPreferences.use_grid = use_grid
Graphene.DomainPreferences.generated_points = generated_points
Graphene.DomainPreferences.convergence = conv
Graphene.DomainPreferences.orientation_iterations = orientation_iterations
Graphene.DomainPreferences.apply_resolution = apply_resolution


runner = EmbeddedLocalRunner()

Running the calculation

In [ ]:
for n in sizes:
    print('Calculating for ' + str(n) + 'x' + str(n) + ' graphene lattice')
    rep_a_xz, rep_b_xz, rep_c_xz = n, 1, n
    dol_out_xz = (r"D:\Eytan\Graphene\For_EPlus_paper\graphene_lattice_1_xz_" + str(rep_a_xz) + '_' + str(rep_c_xz) + r'.dol')
    _ = g.build_crystal(xz, rep_a_xz, rep_b_xz, rep_c_xz, dol_out_xz)

    graphene_unit.read_from_dol(dol_out_xz)

    Graphene.export_all_parameters(r'.\Graphene_' + str(n) + 'x' + str(n) + '_test.state')

    graphene_out = runner.generate(Graphene)
    graphene_out.save_to_out_file(r'.\Graphene_' + str(n) + 'x' + str(n) + '_test.out')
    print('Graphene_' + str(n) + 'x' + str(n) + ' done')

    temp_amp_mono= runner.get_amp(graphene_unit_2.model_ptr)
    temp_amp_bi = runner.get_amp(graphene_2_layer.model_ptr)
    temp_amp_quad = runner.get_amp(graphene_4_layer.model_ptr)

    I_2D_mono += temp_amp_mono.get_crystal_intensity(0, generated_points_2D, q_max) / len(sizes)
    I_2D_bi += temp_amp_bi.get_crystal_intensity(0, generated_points_2D, q_max) / len(sizes)
    I_2D_quad += temp_amp_quad.get_crystal_intensity(0, generated_points_2D, q_max) / len(sizes)

In [ ]:
CR.save_to_2D_out_file(qp, qp, I_2D_mono, f'.\Graphene_2D_mono_av_{sizes[0]}_to_{sizes[-1]}_{generated_points_2D}x{generated_points_2D}'
                                          f'.out2')
CR.save_to_2D_out_file(qp, qp, I_2D_bi, f'.\Graphene_2D_bi_av_{sizes[0]}_to_{sizes[-1]}_{generated_points_2D}x{generated_points_2D}'
                                        f'.out2')
CR.save_to_2D_out_file(qp, qp, I_2D_quad, f'.\Graphene_2D_quad_av_{sizes[0]}_to_{sizes[-1]}_{generated_points_2D}x'
                                          f'{generated_points_2D}.out2')

# Gold NP

In [ ]:
import numpy as np
from scipy.spatial import distance_matrix
import os
import re
import time
import dplus.g_r as g
from dplus.CalculationInput import CalculationInput
from dplus.CalculationRunner import EmbeddedLocalRunner
from dplus.CalculationResult import CalculationResult as calc_res
from dplus.DataModels.models import EPDB, AMP, PDB
from dplus.DataModels import ManualSymmetry
from dplus.Amplitudes import Amplitude
from dplus.EDConverter import different_atoms, convert

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

CGREEN = '\33[32m'
CEND = '\033[0m'

In [ ]:
def extract_number(filename):
    match = re.search(r'_run_(\d+)', filename)
    if match:
        return int(match.group(1))
    return 0  # return 0 if no number found


def read_from_gold_pdb(filename):
    vec = np.array([])
    is_Au = []
    atoms = np.array([])
    n = 0
    with open(filename, encoding='utf-8') as pdb:
        for line in pdb:
            if (line[:6] == 'ATOM  ') | (line[:6] == 'HETATM'):
                n += 1
                atom = line[76:78]
                if (atom == 'AU') or (atom == 'Au'):
                    is_Au.append(True)
                else:
                    is_Au.append(False)

                atoms = np.append(atoms, atom)
                vec = np.append(vec, [float(line[30:38]) / 10, float(line[38:46]) / 10, float(line[46:54]) / 10])
            else:
                continue

    vec = np.reshape(vec, [n, 3])
    return vec, is_Au, atoms


def write_to_gold_pdb(filename, vec, atoms):
    with open(filename, 'w') as pdb:
        for i in range(len(vec)):
            pdb.write('ATOM  {:4d}  {:3s}       {:4d}    {:8.3f}{:8.3f}{:8.3f}  1.00  0.00          {:2s}\n'.format(
                i + 1, atoms[i], 1, vec[i, 0] * 10, vec[i, 1] * 10, vec[i, 2] * 10, atoms[i]))


def write_gold_to_dol(dol_name, au_mat):
    len_mat, w_mat = np.shape(au_mat)
    n = np.reshape(np.linspace(1, len_mat, len_mat), [len_mat, 1])
    zeroes = np.zeros([len_mat, w_mat])
    final_mat = np.concatenate((n, au_mat, zeroes), axis=1)

    np.savetxt(dol_name, final_mat, delimiter='\t')


def add_beamstop(I_2D, qp, qz, beamstop_rad):
    qp, qz = np.meshgrid(qp, qz)
    q = np.sqrt(qp ** 2 + qz ** 2)
    I_2D[q < beamstop_rad] = 0
    return I_2D


def find_sigma(rest_dist):
    return rest_dist / 2 ** (1 / 6)

Defining the parameters

In [ ]:
kJmol_to_kT = 1 / (8.3144621e-3 * 298)
eV_to_kT = 1 / (8.6173324e-5 * 298)

temp = 298
max_dist = 0.45
step_size = 0.01
iterations = int(7.5e4)
rest_dist = 0.32
sigma = 0
pot = 'LJ'
eps_LJ = 0.4129 * eV_to_kT
sig_LJ = 0.268

params_LJ = [eps_LJ, sig_LJ]
pop_num = 100
mini_accept = 2000
max_np_rad = .806

MC_dir = r".\MC"

dol_start = r".\Au144_MD6341surface1_no_S.pdb"
dol_out = MC_dir + r"\DOL\Au144_MD6341_eps_%.3f_sig_%.3f.dol" % (eps_LJ, sig_LJ)

Running MC simulation

In [ ]:
print(CGREEN, 'Starting MC simulation', CEND)

e_vec, d_vec, acc_rate = g.MC_Sim(dol_start, dol_out, temp, max_dist, rest_dist, step_size, iterations, sigma, pot,
                             *params_LJ, pop_out_num=pop_num, min_accepted=mini_accept)

print(CGREEN, 'MC simulation finished, starting to calculate I(q)', CEND)

Parameters for the E+ calculation

In [ ]:
gen_points = 3500
gen_points_2d = 128
qmin = 0
qmax = 70
grid_size = 400
convergence = 0.001
res_sigma = 0.02
update_interval = 1000
thermal_num = 100
garbage = 3
alpha_turn = 0
beta_turn = 35
qp = np.linspace(-qmax, qmax, gen_points_2d)

dols = [filename for filename in os.listdir(MC_dir+ r"\DOL") if filename.startswith("Au144_MD6341_eps_%.3f_sig_%.3f" %
                                                                           (eps_LJ, sig_LJ))]
sorted_dols = sorted(dols, key=extract_number)[garbage:]

iqs_w_ligands = np.zeros([len_dols, gen_points + 1])
iqs_2d_w_ligands = np.zeros([len_dols, gen_points_2d, gen_points_2d])
iq_avg_w_ligands = np.zeros([gen_points + 1])
iq_2d_av_w_ligands = np.zeros([gen_points_2d, gen_points_2d])

ligand_pdb_file = r".\Au144_pMBA_ligands_only.pdb"
ligand_ed_convert = convert(ed=360)

runner = EmbeddedLocalRunner()

Preparing the calculation input

In [ ]:
ligand_pdb = EPDB(ligand_pdb_file)
ligand_pdb.centered = True
ligand_pdb.extra_params.solvent_method.value = 0  # 0 for no solvent

au_pdb = EPDB(r".\Au.pdb")
au_pdb.centered = True
au_pdb.extra_params.solvent_method.value = 0  # 0 for no solvent

symmetry = ManualSymmetry()
symmetry.children.append(au_pdb)

turning_sym = ManualSymmetry()
turning_sym.add_layer()
turning_sym.layer_params[0].alpha.value = alpha_turn
turning_sym.layer_params[0].beta.value = beta_turn
turning_sym.children.append(symmetry)
turning_sym.use_grid = True

calc_input = CalculationInput()
calc_input.DomainPreferences.convergence = convergence
calc_input.DomainPreferences.grid_size = grid_size
calc_input.DomainPreferences.q_max = qmax
calc_input.DomainPreferences.q_min = qmin
calc_input.DomainPreferences.generated_points = gen_points
calc_input.DomainPreferences.update_interval = update_interval
calc_input.DomainPreferences.resolution_sigma = res_sigma
calc_input.DomainPreferences.orientation_iterations = 1e6
calc_input.DomainPreferences.apply_resolution = True
calc_input.DomainPreferences.resolution_sigma = 0.01
calc_input.use_gpu = True
calc_input.DomainPreferences.orientation_method = 'Monte Carlo (Mersenne Twister)'
calc_input.Domain.populations[0].add_model(turning_sym)

Running the E+ calculation

In [ ]:
print(CGREEN, 'Starting to calc. with ligands', CEND)
turning_sym.children.append(ligand_pdb)

for dol_ind, dol in enumerate(sorted_dols):
    print(CGREEN,'\nCalculating dol #{} out of {}: {}'.format(dol_ind+1, len_dols, dol), CEND)
    symmetry.read_from_dol(os.path.join(MC_dir, dol))
    out = runner.generate(calc_input)
    temp_amp = runner.get_amp(turning_sym.model_ptr)

    calc_input.export_all_parameters(os.path.join(MC_dir + '\state', dol[:-4] + '_w_ligands_EP.state'))
    out.save_to_out_file(os.path.join(MC_dir + '\out', dol[:-4] + '_w_ligands_EP.out'))
    iqs_w_ligands[dol_ind, :] = np.array(list(out.graph.values()))
    iqs_2d_w_ligands[dol_ind, :, :] = temp_amp.get_crystal_intensity(qmin, gen_points_2d, qmax)


iq_avg_w_ligands = np.mean(iqs_w_ligands, axis=0)
iq_2d_av_w_ligands = np.mean(iqs_2d_w_ligands, axis=0)

np.savetxt(MC_dir + r"\out\iq_avg_w_ligands_eps_%.3f_sig_%.3f_MC_new_EP.out" % (eps_LJ, sig_LJ),
           np.array([list(out.graph.keys()), iq_avg_w_ligands]).T, delimiter='\t')

calc_res.save_to_2D_out_file(qp, qp, iq_2d_av_w_ligands, MC_dir + r"\out\Yael_Au144_pMBA_eps_%.3f_sig_%"
                                                                  r".3f_2D_new_EP_128.out2" % (eps_LJ, sig_LJ))

print(CGREEN, 'I(q) calculation finished', CEND)

# Microtubules

In [ ]:
import numpy as np
from dplus.CalculationResult import CalculationResult as CR
from dplus.CalculationInput import CalculationInput
from dplus.CalculationRunner import EmbeddedLocalRunner
from dplus.DataModels.models import PDB, EPDB
from dplus.DataModels import ManualSymmetry
import dplus.EDConverter as edc
import matplotlib
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

Defining the parameters

In [ ]:
data_file = r".\ESRF_GTP_Data.dat"
dol_of_Left_MT = [r".\MT\MT_11.05_12.214_13_13_0_16_3_0.0_0.0_0.0.dol",
                  r".\MT\MT_11.9_12.214_14_14_0_16_3_0.0_0.0_0.0.dol",
                  r".\MT\MT_12.75_12.214_15_15_0_16_3_0.0_0.0_0.0.dol"]

q_data, I_data = np.loadtxt(data_file, skiprows=1, unpack=True)

PDB_MT_file = r".\MT\3J6F_GDP_Dimer_2_2_GeometricCenter.pdb"
EPDB_MT_file = r".\MT\3J6F_GDP_Dimer_2_2_GeometricCenter.pdb"

q_min = 0
q_max = 7
grid_size_xray = 120
grid_size_em = 120
orientation_method_mersenne = "Monte Carlo (Mersenne Twister)"
orientation_method_vegas = "Adaptive (VEGAS) Monte Carlo"
use_grid = True
generated_points = q_max * 100
conv = 1e-3
orientation_iterations = 1e6
apply_resolution = True
resolution = 0.01

H2O_convert = edc.convert(333, a=['H', 'O'], n=[2, 1])
H2O_eED = H2O_convert.eED
H2O_coeff = H2O_convert.coeff
print(f"eED of H2O: {H2O_eED}", f"coeff of H2O: {H2O_coeff}")

Preparing the MT state

In [ ]:
MT_PDB = PDB(PDB_MT_file)
MT_PDB.centered = True
MT_PDB.location_params.gamma.value = 271.5 * np.pi / 180
MT_PDB.extra_params.solvent_voxel_size.value = 0.05
MT_PDB.extra_params.solvent_method.value = 5

MT_EPDB = EPDB(EPDB_MT_file)
MT_EPDB.centered = True
MT_EPDB.location_params.gamma.value = 271.5 * np.pi / 180
MT_EPDB.extra_params.solvent_ed.value = H2O_eED
MT_EPDB.extra_params.solvent_voxel_size.value = 0.05
MT_EPDB.extra_params.solvent_method.value = 5


MT_struct_xray = ManualSymmetry()
MT_struct_xray.location_params.y.value = -11.846
MT_struct_xray.location_params.z.value = -9.2129
# MT_struct_xray.read_from_dol(dol_of_Left_MT)
MT_struct_xray.children.append(MT_PDB)
MT_struct_xray.use_grid = False

MT_struct_EM = ManualSymmetry()
MT_struct_EM.location_params.y.value = -11.846
MT_struct_EM.location_params.z.value = -9.2129
# MT_struct_EM.read_from_dol(dol_of_Left_MT)
MT_struct_EM.children.append(MT_EPDB)
MT_struct_EM.use_grid = False

MT_CalcInput_xray = CalculationInput()
MT_CalcInput_xray.Domain.populations[0].add_model(MT_struct_xray)
# MT_CalcInput_xray.DomainPreferences.q_max = q_max
MT_CalcInput_xray.DomainPreferences.grid_size = grid_size_xray
MT_CalcInput_xray.DomainPreferences.orientation_method = orientation_method_vegas
MT_CalcInput_xray.DomainPreferences.use_grid = use_grid
# MT_CalcInput_xray.DomainPreferences.generated_points = generated_points
MT_CalcInput_xray.DomainPreferences.convergence = conv
MT_CalcInput_xray.DomainPreferences.orientation_iterations = orientation_iterations
MT_CalcInput_xray.DomainPreferences.apply_resolution = apply_resolution
MT_CalcInput_xray.DomainPreferences.resolution_sigma = resolution
MT_CalcInput_xray.DomainPreferences.update_interval = 1000
MT_CalcInput_xray.use_gpu = True
MT_CalcInput_xray.DomainPreferences.signal_file = data_file

MT_CalcInput_EM = CalculationInput()
MT_CalcInput_EM.Domain.populations[0].add_model(MT_struct_EM)
MT_CalcInput_EM.DomainPreferences.q_max = q_max
MT_CalcInput_EM.DomainPreferences.grid_size = grid_size_xray
MT_CalcInput_EM.DomainPreferences.orientation_method = orientation_method_vegas
MT_CalcInput_EM.DomainPreferences.use_grid = use_grid
MT_CalcInput_EM.DomainPreferences.generated_points = generated_points
MT_CalcInput_EM.DomainPreferences.convergence = conv
MT_CalcInput_EM.DomainPreferences.orientation_iterations = orientation_iterations
MT_CalcInput_EM.DomainPreferences.apply_resolution = apply_resolution
MT_CalcInput_EM.DomainPreferences.resolution_sigma = resolution
MT_CalcInput_EM.DomainPreferences.update_interval = 1000
MT_CalcInput_EM.use_gpu = True
MT_CalcInput_EM.DomainPreferences.signal_file = data_file


runner = EmbeddedLocalRunner()

Running the calculation

In [ ]:
xray_results = np.zeros([3, MT_CalcInput_xray.DomainPreferences.generated_points+1])
em_results = np.zeros([3, MT_CalcInput_EM.DomainPreferences.generated_points+1])

for i in range(0, 3):
    print(f"Running X-ray {i+13}")
    MT_struct_xray.read_from_dol(dol_of_Left_MT[i])
    MT_CalcOutput_xray = runner.generate(MT_CalcInput_xray)
    xray_results[i] = np.array(list(MT_CalcOutput_xray.graph.values()))
    MT_CalcOutput_xray.save_to_out_file(f".\MT\MT_CalcOutput_xray_{i+13}_pf_new_16ppf.out")
    MT_CalcInput_xray.export_all_parameters(f".\MT\Tubulin_xray_{i+13}_pf_new_16ppf.state")

    print(f"Running EM {i+13}")
    MT_struct_EM.read_from_dol(dol_of_Left_MT[i])
    MT_CalcOutput_EM = runner.generate(MT_CalcInput_EM)
    em_results[i] = np.array(list(MT_CalcOutput_EM.graph.values()))
    MT_CalcOutput_EM.save_to_out_file(f".\MT\MT_CalcOutput_EM_{i+13}_pf_new_16ppf.out")
    MT_CalcInput_EM.export_all_parameters(f".\MT\Tubulin_EM_{i+13}_pf_for_2d_16ppf.state")

mass_fraction = np.array([0.2, 0.7, 0.1])

xray_results_all = np.sum(xray_results * mass_fraction[:, np.newaxis], axis=0)
em_results_all = np.sum(em_results * mass_fraction[:, np.newaxis], axis=0)

np.savetxt(r".\MT\MT_CalcOutput_xray_new_16.out", np.array([list(MT_CalcOutput_xray.graph.keys()),
                                                                  xray_results_all]).T)
np.savetxt(r".\MT\MT_CalcOutput_EM_new_16.out", np.array([list(MT_CalcOutput_EM.graph.keys()),
                                                                    em_results_all]).T)

Calculating the fibre diffraction

In [ ]:
MT_fiber_EM = CalculationInput(is2D=True)
MT_fiber_EM.Domain.populations[0].add_model(MT_struct_EM)
MT_fiber_EM.DomainPreferences.q_max = 5
MT_fiber_EM.DomainPreferences.grid_size = grid_size_xray
MT_fiber_EM.DomainPreferences.orientation_method = orientation_method_mersenne
MT_fiber_EM.DomainPreferences.use_grid = use_grid
MT_fiber_EM.DomainPreferences.generated_points = 250
MT_fiber_EM.DomainPreferences.convergence = conv
MT_fiber_EM.DomainPreferences.orientation_iterations = 1e6
MT_fiber_EM.DomainPreferences.apply_resolution = apply_resolution
MT_fiber_EM.DomainPreferences.resolution_sigma = resolution
MT_fiber_EM.DomainPreferences.update_interval = 1000
MT_fiber_EM.use_gpu = False

MT_struct_EM.read_from_dol(dol_of_Left_MT[1])

MT_14_out2D = runner.generate2D(MT_fiber_EM)

qp = np.array(list(MT_14_out2D.graph.keys()))
I_14_fibre = np.array(list(MT_14_out2D.graph.values()))

MT_14_out2D.save_to_2D_out_file(qp , qp, I_14_fibre, r".\MT\MT_CalcOutput_EM_14pf_16ppf_fibre.out2")